In [1]:
## Libraries Installation
### NLTK Installation
%pip install nltk
import nltk
nltk.download('all')
### ir_datasets Installation
# %pip install --user ir_datasets

### PyQt6 Installation
%pip install PyQt6
### rank_bm25 Installation
%pip install rank_bm25
### joblib Installation
%pip install joblib
%pip install dill
## Apply ir_datasets fix for encoding
%pip install contractions
import os
%pip install chromadb

Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\FSOS\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\FSOS\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\FSOS\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\FSOS\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\FSOS\AppData\Roaming\nltk_data...
[nltk_

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
    from sentence_transformers import SentenceTransformer
    gpu_model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')

c:\Users\FSOS\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from sentence_transformers import SentenceTransformer
import torch

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')
model = model.to('cuda')  # Force move to GPU

# When encoding, ensure device is set
texts = ["query " + str(i) for i in range(1000)]
embeddings = model.encode(texts, device='cuda', batch_size=64, show_progress_bar=True)

Batches: 100%|██████████| 16/16 [00:00<00:00, 32.29it/s]


In [4]:
%pip install ir_datasets
import ir_datasets
import ir_datasets

from typing import TypeAlias

from collections import namedtuple

Doc = namedtuple('Doc', ['doc_id', 'text'])
Query = namedtuple('Query', ['query_id', 'text'])
Qrel = namedtuple('Qrel', ['query_id', 'doc_id', 'relevance', 'iteration'])

def load_dataset(name: str) -> list[Doc]:
    dataset = ir_datasets.load(DATASETS[name]['ir_datasets_id'])

    docs = list(dataset.docs_iter())
    return docs

def load_queries_and_qrels(name: str) -> tuple[list[Query], list[Qrel]]:
    dataset_test = ir_datasets.load(DATASETS[name]['ir_datasets_test_id'])

    queries = list(dataset_test.queries_iter())
    qrels = list(dataset_test.qrels_iter())
    return queries, qrels

def load_dataset_with_queries(name: str) -> tuple[list[Doc], list[Query], list[Qrel]]:
    dataset = ir_datasets.load(DATASETS[name]['ir_datasets_id'])
    dataset_test = ir_datasets.load(DATASETS[name]['ir_datasets_test_id'])

    docs = list(dataset.docs_iter())
    queries = list(dataset_test.queries_iter())
    qrels = list(dataset_test.qrels_iter())

    return docs, queries, qrels

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Dataset configurations
DATASETS = {
    "antique": {
        "name": "antique",
        "description": "Question-answer dataset with natural questions from real users",
        "ir_datasets_id": "antique",
        "ir_datasets_test_id": "antique/test/non-offensive"
    },
    "quora": {
        "name": "beir/quora",
        "description": "Quora question pairs dataset from the BEIR benchmark",
        "ir_datasets_id": "beir/quora",
        "ir_datasets_test_id": "beir/quora/dev"
    },
}

# Default dataset to use if none specified
DEFAULT_DATASET = "antique"


In [6]:
dataset_name = 'antique'

# Load the dataset using the loader
docs= load_dataset(dataset_name)


In [7]:
import re
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
import contractions

class TextPreprocessor:
    __lemmatizer__ = WordNetLemmatizer()
    __stop_words__ = set(stopwords.words('english'))
    __instance__ = None

    @staticmethod
    def getInstance():
        if TextPreprocessor.__instance__ == None:
            TextPreprocessor.__instance__ = TextPreprocessor()
        return TextPreprocessor.__instance__

    def __clean_text__(self, text):
        text = contractions.fix(text)
        text = text.lower()
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        return text

    def __remove_stopwords__(self, text):
        words = word_tokenize(text)
        filtered_words = [word for word in words if word not in self.__stop_words__]
        return ' '.join(filtered_words)

    def __get_wordnet_pos__(self, tag_parameter):
        tag = tag_parameter[0].upper()
        tag_dict = {
            "J": wordnet.ADJ,
            "N": wordnet.NOUN,
            "V": wordnet.VERB,
            "R": wordnet.ADV
            }
        return tag_dict.get(tag, wordnet.NOUN)

    def __lemmatize_text__(self, text):
        words = word_tokenize(text)
        pos_tags = pos_tag(words)
        lemmatized_words = [self.__lemmatizer__.lemmatize(word, pos=self.__get_wordnet_pos__(tag)) for word, tag in pos_tags]
        return ' '.join(lemmatized_words)

    def preprocess_text(self, text, remove_stopwords_flag=True): 
        if not isinstance(text, str):
            return [] 
        
        text = self.__clean_text__(text)
        text = self.__lemmatize_text__(text)
        
        if remove_stopwords_flag:
            text = self.__remove_stopwords__(text)

        return word_tokenize(text.strip())

In [8]:
import math

def calc_dcg(relevance, rank):
    return ((2 ** relevance) - 1) / math.log10(rank + 1)

class Retriever:
    def search(self, dataset_name: str, query: str, top_k: int = 10, with_index: bool = True) -> list[tuple[str, float, str]]:
        raise NotImplementedError()

    def evaluateNDCG(self, dataset_name, queries, qrels, docs, K = 10, print_more = False, with_index: bool = True):
        nDCG = []

        for i in range(len(queries)):
            query = queries[i]
            if print_more:
                preprocess_text = TextPreprocessor.getInstance().preprocess_text
                print(f"Query: {query.text}")
                print(f"Query: {preprocess_text(query.text)}")

            # Search
            results = self.search(dataset_name, query.text, K, with_index)
            if print_more:
                for i, res in enumerate(results):
                    print(f"Result #{i} {res[1]}: {res[2]}")
                    print(f"Result #{i} {res[1]}: {preprocess_text(res[2])}")

            # Find relevant documents for this query
            relevant_qrels = [qrel for qrel in qrels if qrel.query_id == query.query_id]
            relevant_qrels = sorted(relevant_qrels, key=lambda x: x.relevance, reverse=True)
            if print_more:
                for i, qrel in enumerate(relevant_qrels[:K]):
                    doc = [doc for doc in docs if qrel.doc_id == doc.doc_id][0]
                    print(f"Qrel #{i} {qrel.relevance}: {doc.text}")
                    print(f"Qrel #{i} {qrel.relevance}: {preprocess_text(doc.text)}")

            DCG = [
                calc_dcg(
                    list(
                        filter(
                            lambda qrel: qrel.doc_id == doc[0], relevant_qrels
                            )
                        )[0].relevance if list(
                        filter(
                            lambda qrel: qrel.doc_id == doc[0], relevant_qrels
                            )
                        ) else 0
                    , i+1
                ) for i, doc in enumerate(results)]

            iDCG = [calc_dcg(qrel.relevance, i+1) for i, qrel in enumerate(relevant_qrels[:K])]

            res = sum(DCG)
            ires = sum(iDCG)

            if print_more:
                print("")
                print(f"query: {i+1}/{len(queries)}")
                print(f"DCG: {res}")
                print(f"iDCG: {ires}")
                print(f"nDCG: {res/ires*100}%")
            nDCG.append(res/ires)
            if print_more:
                print(f"Average nDCG: {sum(nDCG)/len(nDCG)*100}%")

        nDCG = sum(nDCG)/len(nDCG)*100

        if print_more:
            print(f"Final Average nDCG: {nDCG}%")

        return nDCG

    def evaluateMRR(self, dataset_name, queries, qrels, K = 100, print_more = False):
        MRR = []

        cleaned_qrels: dict[str, dict[str, int]] = {}
        for qrel in qrels:
            if qrel.query_id not in cleaned_qrels.keys():
                cleaned_qrels[qrel.query_id] = {}
            cleaned_qrels[qrel.query_id][qrel.doc_id] = qrel.relevance

        for i in range(len(queries)):
            query = queries[i]
            results = self.search(dataset_name, query.text, K, True)

            firstRank = 100
            for ii, res in enumerate(results):
                if res[0] in cleaned_qrels[query.query_id].keys() and cleaned_qrels[query.query_id][res[0]] > 0:
                    firstRank = ii + 1

            MRR.append(1/firstRank)

            if print_more:
                print()
                print(f"Query: {i+1}/{len(queries)}")
                print(f"Current MRR: {sum(MRR) / len(MRR) * 100}")

        MRR = sum(MRR) / len(MRR) * 100
        if print_more:
            print(f"MRR: {MRR}%")
        return MRR

    def evaluateMAP(self, dataset_name, queries, qrels,docs, K = 10, print_more = False):
        MAP = []

        cleaned_qrels: dict[str, dict[str, int]] = {}
        for qrel in qrels:
            if qrel.query_id not in cleaned_qrels.keys():
                cleaned_qrels[qrel.query_id] = {}
            cleaned_qrels[qrel.query_id][qrel.doc_id] = qrel.relevance

        for i in range(len(queries)):
            query = queries[i]
            if print_more:
                print()
                print(f'Query: {i+1}/{len(queries)}')
                print(query.text)

            results = self.search(dataset_name, query.text, K, True)
            if print_more:
                print([res[0] for res in results])
                print([qrel.doc_id+f": {qrel.relevance}" for qrel in qrels if qrel.query_id == query.query_id])
                print("results")
                for doc in [doc for doc in docs if doc.doc_id in [res[0] for res in results]]:
                    print(doc.doc_id+" "+doc.text)
                print("qrels")
                koko = [qrel.doc_id for qrel in qrels if qrel.query_id == query.query_id]
                for doc in [doc for doc in docs if doc.doc_id in koko]:
                    print(doc.doc_id+" "+doc.text)

            relevant_num = 0
            precision_sum = 0
            for ii, res in enumerate(results):
                if res[0] in cleaned_qrels[query.query_id].keys() and cleaned_qrels[query.query_id][res[0]] > 0:
                    relevant_num += 1
                    precision_sum += relevant_num / (ii + 1)
            if print_more:
                print(precision_sum)
            if relevant_num > 0:
                MAP.append(precision_sum / relevant_num)
            if print_more:
                if len(MAP) > 0:
                    print(f'MAP = {sum(MAP) / len(MAP) * 100}')
        if len(MAP) > 0:
            MAP = sum(MAP) / len(MAP) * 100
        else:
            MAP = 0
        if print_more:
            print(f'MAP={MAP}%')
        return MAP


In [10]:
import math
import chromadb
from sentence_transformers import SentenceTransformer, util
import numpy as np
import torch 

class Embedding_online(Retriever):
    __embeddingInstance__ : dict[str, any] = {}
    __collection_instance__: dict = {}
    __modelInstance__ = None

    @staticmethod
    def __loadModelInstance__():
        if Embedding_online.__modelInstance__ == None:
            # **مسار الموديل محلياً:** سيتم تنزيله إلى الكاش الافتراضي لـ SentenceTransformer (~/.cache/torch/sentence_transformers)
            # إذا كنت تريد حفظه في مكان معين داخل مشروعك:
            # Embedding_online.__modelInstance__ = SentenceTransformer("./data/models/all-MiniLM-L6-v2")
            Embedding_online.__modelInstance__ = SentenceTransformer("all-MiniLM-L6-v2") 
            
            # **هام: انقل الموديل إلى الـ GPU بعد التحميل**
            if torch.cuda.is_available():
                Embedding_online.__modelInstance__.to('cuda')
                print("Embedding model moved to GPU.")
            else:
                print("GPU not available, embedding model running on CPU.")
                
        return Embedding_online.__modelInstance__

    @staticmethod
    def __loadInstance__(dataset_name : str):
        # **هام: تأكد أن هذا المسار صحيح لملفات .npy على جهازك**
        if dataset_name not in Embedding_online.__embeddingInstance__.keys():
            # افترض أن 'data' موجود في جذر المشروع
            with open(f"./data/{dataset_name}/bert_embeddings.npy", "rb") as f:
                Embedding_online.__embeddingInstance__[dataset_name] = np.load(f)
        return Embedding_online.__embeddingInstance__[dataset_name]

    @staticmethod
    def __get_collection__(dataset_name: str):
        # **هام: تأكد أن هذا المسار صحيح لمجلد ChromaDB للوثائق (إذا كنت تستخدمه)**
        if dataset_name not in Embedding_online.__collection_instance__:
            print(f"Connecting to ChromaDB and getting collection: {dataset_name}_embeddings...")
            # افترض أن 'chroma_db' موجود في جذر المشروع
            client = chromadb.PersistentClient(path="./chroma_db") 
            Embedding_online.__collection_instance__[dataset_name] = client.get_collection(name=f"{dataset_name}_embeddings")
        return Embedding_online.__collection_instance__[dataset_name]

    def search(self, dataset_name: str, query: str, top_k: int = 10, with_index: bool = True) -> list[tuple[str, float, str]]:
        if with_index:
            return self.embedding_vectors_search(dataset_name, query, top_k)
        else:
            return self.embedding_search(dataset_name, query, top_k)

    def embedding_search(self, dataset_name: str, query: str, top_k: int):
        model = Embedding_online.__loadModelInstance__() 
        document_embeddings = Embedding_online.__loadInstance__(dataset_name)
        docs = load_dataset(dataset_name)
        
        processedQueryTokens = TextPreprocessor.getInstance().preprocess_text(query, remove_stopwords_flag=False)
        processedQuery = " ".join(processedQueryTokens) 
        
        query_embedding = model.encode(processedQuery, convert_to_tensor=True) 
        
        cos_scores = util.cos_sim(query_embedding, torch.tensor(document_embeddings).to(query_embedding.device))[0]
        top_results = torch.topk(cos_scores, k=top_k)
        results = []
        for score, idx in zip(top_results[0], top_results[1]):
            doc_id = docs[idx].doc_id
            doc_text = docs[idx].text[:100] + "..." 
            results.append((doc_id, score.item(), doc_text))
        return results

    def embedding_vectors_search(self, dataset_name: str, query: str, top_k: int):
        model = Embedding_online.__loadModelInstance__() 
        collection = Embedding_online.__get_collection__(dataset_name)
        
        processedQueryTokens = TextPreprocessor.getInstance().preprocess_text(query, remove_stopwords_flag=False)
        processedQuery = " ".join(processedQueryTokens) 
        
        query_embedding = model.encode(processedQuery, convert_to_tensor=False) 
        search_results = collection.query(
            query_embeddings=[query_embedding],
            n_results=top_k
        )
        results = []
        ids = search_results['ids'][0]
        distances = search_results['distances'][0]
        metadatas = search_results['metadatas'][0]
        for doc_id, score, meta in zip(ids, distances, metadatas):
            similarity_score = 1 - score
            text = meta.get('text', '')[:100] + "..." 
            results.append((doc_id, similarity_score, text))
        return results

    def embedding_rerank(self, dataset_name: str, query: str, doc_ids: list) -> list[tuple[str, float]]:
        docs_list = load_dataset(dataset_name)
        model = Embedding_online.__loadModelInstance__() 

        doc_texts_to_rerank = []
        doc_id_to_original_doc_obj = {doc.doc_id: doc for doc in docs_list} 
        
        for doc_id in doc_ids:
            if doc_id in doc_id_to_original_doc_obj:
                doc_texts_to_rerank.append(doc_id_to_original_doc_obj[doc_id].text)
            
        if not doc_texts_to_rerank:
            return []

        candidate_embeddings = model.encode(doc_texts_to_rerank, convert_to_tensor=True) 
        
        query_embedding = model.encode(query, convert_to_tensor=True)
        
        cosine_scores = util.cos_sim(query_embedding, candidate_embeddings)[0]
        
        reranked_results = []
        for i, score in enumerate(cosine_scores):
            reranked_results.append((doc_ids[i], score.item()))

        return sorted(reranked_results, key=lambda item: item[1], reverse=True)

In [12]:
import os
import joblib
from collections import Counter
import numpy as np 
import chromadb 
from uuid import uuid4 
import gc # استيراد مجمع القمامة


INDEX_SAVE_PATH_BASE = "./data" # افترض أن 'data' في جذر المشروع
# **مسار ChromaDB لاقتراحات الكويري:**
N_GRAM_CHROMADB_PATH = "./chroma_db_suggestions" # افترض أن 'chroma_db_suggestions' في جذر المشروع

class QuerySuggestionService:
    def __init__(self, dataset_name, preprocessor: TextPreprocessor):
        self.dataset_name = dataset_name
        self.preprocessor = preprocessor
        self.embedding_model = Embedding_online.__loadModelInstance__() 
        
        self.all_docs_by_id = {}
        self.all_queries_by_id = {}

        self.chroma_client = chromadb.PersistentClient(path=N_GRAM_CHROMADB_PATH)
        self.suggestions_collection_name = f"{dataset_name.replace('/', '_')}_ngrams"
        
        self._load_or_build_index()

    def _get_index_file_path(self):
        return os.path.join(INDEX_SAVE_PATH_BASE, self.dataset_name.replace('/', '_'), "query_suggestion_chromadb_indexed.flag")

    def _load_index(self):
        flag_path = self._get_index_file_path()
        if os.path.exists(flag_path):
            try:
                self.suggestions_collection = self.chroma_client.get_collection(name=self.suggestions_collection_name)
                print(f"Query suggestion ChromaDB collection '{self.suggestions_collection_name}' loaded successfully.")
                return True
            except Exception as e:
                print(f"Failed to load ChromaDB collection: {e}. Rebuilding index.")
                return False
        return False

    def _build_index(self):
        print(f"Building query suggestion index with embeddings for {self.dataset_name} in ChromaDB...")
        
        try:
            self.suggestions_collection = self.chroma_client.get_or_create_collection(name=self.suggestions_collection_name)
        except Exception as e:
            print(f"Error creating/getting ChromaDB collection: {e}. Aborting index build.")
            return

        existing_ids = self.suggestions_collection.get(limit=1)['ids'] 
        if existing_ids: 
            print(f"ChromaDB collection '{self.suggestions_collection_name}' has existing data. Clearing...")
            self.suggestions_collection.delete(where={}) 
            print(f"Cleared existing data in ChromaDB collection '{self.suggestions_collection_name}'.")
        else:
            print(f"ChromaDB collection '{self.suggestions_collection_name}' is already empty. No data to clear.")


        try:
            docs_list = load_dataset(self.dataset_name) 
            queries_list, _ = load_queries_and_qrels(self.dataset_name)
        except KeyError:
            print(f"Dataset '{self.dataset_name}' not found in DATASETS configuration.")
            return

        self.all_docs_by_id = {doc.doc_id: doc.text for doc in docs_list}
        self.all_queries_by_id = {query.query_id: query.text for query in queries_list}

        all_text_sources = []
        
        if not docs_list:
            print(f"Warning: No documents loaded from ir_datasets for {self.dataset_name}. Suggestions will be limited for document-based content.")
        for doc in docs_list:
            if isinstance(doc.text, str):
                all_text_sources.append((doc.text, doc.doc_id, 'doc'))

        if not queries_list:
             print(f"Warning: No queries loaded from ir_datasets for {self.dataset_name}. Suggestions will be limited for query-based content.")
        for query_obj in queries_list:
            if isinstance(query_obj.text, str):
                all_text_sources.append((query_obj.text, query_obj.query_id, 'query'))
        
        if not all_text_sources:
            print(f"No text data available to build query suggestion index for {self.dataset_name}.")
            return

        N_GRAM_PROCESSING_BATCH_SIZE = 5000 
        MAX_SOURCES_PER_NGRAM = 10 
        CHROMA_ADD_SUB_BATCH_SIZE = 200 

        total_ngrams_added = 0

        for i in range(0, len(all_text_sources), N_GRAM_PROCESSING_BATCH_SIZE):
            batch_texts_with_source_info = all_text_sources[i:i + N_GRAM_PROCESSING_BATCH_SIZE]
            
            current_batch_ngrams_meta = {} 

            for text, source_id, source_type in batch_texts_with_source_info:
                processed_tokens = self.preprocessor.preprocess_text(text, remove_stopwords_flag=False) 
                
                for token in processed_tokens:
                    gram_text = token
                    meta = current_batch_ngrams_meta.setdefault(gram_text, {'sources': set(), 'count': 0})
                    if len(meta['sources']) < MAX_SOURCES_PER_NGRAM:
                        meta['sources'].add((source_id, source_type))
                    meta['count'] += 1 

                if len(processed_tokens) > 1:
                    for j in range(len(processed_tokens) - 1):
                        gram_text = " ".join((processed_tokens[j], processed_tokens[j+1]))
                        meta = current_batch_ngrams_meta.setdefault(gram_text, {'sources': set(), 'count': 0})
                        if len(meta['sources']) < MAX_SOURCES_PER_NGRAM:
                            meta['sources'].add((source_id, source_type))
                        meta['count'] += 1

                if len(processed_tokens) > 2:
                    for j in range(len(processed_tokens) - 2):
                        gram_text = " ".join((processed_tokens[j], processed_tokens[j+1], processed_tokens[j+2]))
                        meta = current_batch_ngrams_meta.setdefault(gram_text, {'sources': set(), 'count': 0})
                        if len(meta['sources']) < MAX_SOURCES_PER_NGRAM:
                            meta['sources'].add((source_id, source_type))
                        meta['count'] += 1
            
            batch_ngrams_texts = []
            batch_ngrams_ids = []
            batch_ngrams_metadatas = []

            for gram_text, meta in current_batch_ngrams_meta.items():
                batch_ngrams_texts.append(gram_text)
                batch_ngrams_ids.append(str(uuid4()))
                source_refs_str = ",".join([f"{sid}:{stype}" for sid, stype in meta['sources']])
                batch_ngrams_metadatas.append({'text': gram_text, 'sources': source_refs_str, 'count': meta['count']}) 

            if not batch_ngrams_texts:
                print(f"Skipping empty N-gram batch {i//N_GRAM_PROCESSING_BATCH_SIZE + 1}.")
                continue

            print(f"Generating embeddings for {len(batch_ngrams_texts)} N-grams in batch {i//N_GRAM_PROCESSING_BATCH_SIZE + 1}...")
            n_gram_embeddings = self.embedding_model.encode(batch_ngrams_texts, convert_to_tensor=False, device='cuda') 

            for j in range(0, len(batch_ngrams_ids), CHROMA_ADD_SUB_BATCH_SIZE):
                sub_batch_ids = batch_ngrams_ids[j:j+CHROMA_ADD_SUB_BATCH_SIZE]
                sub_batch_embeddings = n_gram_embeddings[j:j+CHROMA_ADD_SUB_BATCH_SIZE]
                sub_batch_metadatas = batch_ngrams_metadatas[j:j+CHROMA_ADD_SUB_BATCH_SIZE]
                
                self.suggestions_collection.add(
                    embeddings=sub_batch_embeddings.tolist(),
                    metadatas=sub_batch_metadatas,
                    ids=sub_batch_ids
                )
                total_ngrams_added += len(sub_batch_ids)
            print(f"Processed source batch {i//N_GRAM_PROCESSING_BATCH_SIZE + 1}. Total N-grams added so far: {total_ngrams_added}")
            
            del batch_texts_with_source_info
            del current_batch_ngrams_meta
            del batch_ngrams_texts
            del batch_ngrams_ids
            del batch_ngrams_metadatas
            del n_gram_embeddings 
            gc.collect()

        print(f"Finished processing all text sources. Total N-grams added to ChromaDB: {total_ngrams_added}")
        
        os.makedirs(os.path.dirname(self._get_index_file_path()), exist_ok=True)
        with open(self._get_index_file_path(), 'w') as f:
            f.write("indexed")
        print("Query suggestion index built and saved successfully in ChromaDB.")


    def _load_or_build_index(self):
        if not self._load_index():
            self._build_index()

    def _extract_snippet(self, full_text: str, phrase: str, window_size: int = 50) -> str:
        if not full_text or not phrase:
            return ""
        
        full_text_lower = full_text.lower()
        phrase_lower = phrase.lower()

        try:
            start_index = full_text_lower.index(phrase_lower)
            end_index = start_index + len(phrase_lower)

            snippet_start = max(0, start_index - window_size)
            snippet_end = min(len(full_text), end_index + window_size)

            snippet = full_text[snippet_start:snippet_end]
            
            if snippet_start > 0:
                snippet = "..." + snippet
            if snippet_end < len(full_text):
                snippet = snippet + "..."
            
            return snippet.replace('\n', ' ').strip() 
        except ValueError:
            return full_text[:window_size * 2].replace('\n', ' ').strip() + "..." if len(full_text) > window_size * 2 else full_text.replace('\n', ' ').strip()


    def get_suggestions(self, query_prefix: str, top_k: int = 10) -> list[tuple[str, str]]:
        if not hasattr(self, 'suggestions_collection') or self.suggestions_collection is None:
            print("ChromaDB suggestions collection not initialized. Please rebuild index.")
            return []

        processed_prefix_tokens = self.preprocessor.preprocess_text(query_prefix, remove_stopwords_flag=False)
        processed_prefix = " ".join(processed_prefix_tokens)

        if not processed_prefix:
            return []

        query_embedding = self.embedding_model.encode(processed_prefix, convert_to_tensor=False, device='cuda') 

        search_results = self.suggestions_collection.query(
            query_embeddings=[query_embedding.tolist()], 
            n_results=top_k * 5, 
            include=['metadatas', 'distances']
        )
        
        candidate_suggestions_with_details = []

        if search_results and search_results['ids'] and search_results['metadatas']:
            for i in range(len(search_results['ids'][0])):
                gram_text = search_results['metadatas'][0][i]['text']
                source_refs_str = search_results['metadatas'][0][i]['sources'] 
                distance = search_results['distances'][0][i]
                similarity = 1 - (distance / (self.embedding_model.get_sentence_embedding_dimension()**0.5)) if self.embedding_model.get_sentence_embedding_dimension() else 1 - distance 

                source_references = set()
                for ref_str in source_refs_str.split(','):
                    if ref_str:
                        try:
                            sid, stype = ref_str.split(':', 1)
                            source_references.add((sid, stype))
                        except ValueError:
                            pass 

                candidate_suggestions_with_details.append((gram_text, similarity, source_references))

        candidate_suggestions_with_details.sort(key=lambda x: x[1], reverse=True)
        
        final_suggestions_with_snippets = []
        seen_suggestions = set()
        
        for suggestion_text, _, source_references in candidate_suggestions_with_details:
            if suggestion_text not in seen_suggestions:
                snippet = ""
                if source_references:
                    source_id, source_type = next(iter(source_references)) 
                    
                    original_text = ""
                    if source_type == 'doc' and source_id in self.all_docs_by_id:
                        original_text = self.all_docs_by_id[source_id]
                    elif source_type == 'query' and source_id in self.all_queries_by_id:
                        original_text = self.all_queries_by_id[source_id]

                    if original_text:
                        snippet = self._extract_snippet(original_text, suggestion_text)
                
                final_suggestions_with_snippets.append((suggestion_text, snippet))
                seen_suggestions.add(suggestion_text)
            if len(final_suggestions_with_snippets) >= top_k:
                break
        
        return final_suggestions_with_snippets

In [13]:
    from sentence_transformers import SentenceTransformer
    import torch

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
    # Now all encoding will use GPU if available

In [14]:
  import torch
  print(torch.cuda.is_available())  # Should be True
  print(torch.cuda.current_device())
  print(torch.cuda.get_device_name(0))

True
0
NVIDIA GeForce RTX 4050 Laptop GPU


In [15]:
# src/main.py
# ... (استيرادات أخرى)

# ... (في مكان تهيئة الخدمات)

# مثال على تهيئة المعالج المسبق
text_preprocessor_instance = TextPreprocessor.getInstance() # قم بتهيئة المعالج الخاص بك

# تهيئة خدمة اقتراح الكويري لكل مجموعة بيانات
# افترض أن لديك قائمة بأسماء مجموعات البيانات
dataset_names = ["antique", "quora"] # مثال لأسماء مجموعات البيانات الخاصة بك
query_suggestion_services = {}

for dataset_name in dataset_names:
    query_suggestion_services[dataset_name] = QuerySuggestionService(dataset_name, text_preprocessor_instance)
for service in query_suggestion_services.values():
    service.embedding_model = gpu_model
    # Patch all QuerySuggestionService instances to use the GPU-enabled embedding model


# ... (بقية منطق main.py)

# يمكنك الآن الوصول إلى خدمة الاقتراح لمجموعة بيانات معينة
# على سبيل المثال، إذا كنت تعمل على 'msmarco-passage'
# current_dataset_suggestion_service = query_suggestion_services["msmarco-passage"]
# suggestions = current_dataset_suggestion_service.get_suggestions("example qu")

Embedding model moved to GPU.
Building query suggestion index with embeddings for antique in ChromaDB...
ChromaDB collection 'antique_ngrams' is already empty. No data to clear.
Generating embeddings for 267390 N-grams in batch 1...
Processed source batch 1. Total N-grams added so far: 267390
Generating embeddings for 263142 N-grams in batch 2...
Processed source batch 2. Total N-grams added so far: 530532
Generating embeddings for 260719 N-grams in batch 3...
Processed source batch 3. Total N-grams added so far: 791251
Generating embeddings for 266600 N-grams in batch 4...
Processed source batch 4. Total N-grams added so far: 1057851
Generating embeddings for 252892 N-grams in batch 5...
Processed source batch 5. Total N-grams added so far: 1310743
Generating embeddings for 249272 N-grams in batch 6...
Processed source batch 6. Total N-grams added so far: 1560015
Generating embeddings for 246503 N-grams in batch 7...
Processed source batch 7. Total N-grams added so far: 1806518
Genera

KeyboardInterrupt: 

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

True
0
NVIDIA GeForce RTX 4050 Laptop GPU


In [ ]:
import sentence_transformers
print(sentence_transformers.__version__)

4.1.0


In [ ]:
pip install huggingface_hub[hf_xet]

Note: you may need to restart the kernel to use updated packages.


In [ ]:

# Ensure DATASETS is defined if running this cell independently
DATASETS = {
    "antique": {
        "name": "antique",
        "description": "Question-answer dataset with natural questions from real users",
        "ir_datasets_id": "antique",
        "ir_datasets_test_id": "antique/test/non-offensive"
    },
    "quora": {
        "name": "beir/quora",
        "description": "Quora question pairs dataset from the BEIR benchmark",
        "ir_datasets_id": "beir/quora",
        "ir_datasets_test_id": "beir/quora/dev"
    },
}

try:
    docs_test = load_dataset('antique')
    print(f"Loaded {len(docs_test)} documents for antique.")
    if len(docs_test) > 0:
        print(f"First document ID: {docs_test[0].doc_id}")
        print(f"First document text (snippet): {docs_test[0].text[:200]}...")
    else:
        print("load_dataset('antique') returned an empty list.")
except Exception as e:
    print(f"Error loading dataset: {e}")

Loaded 403666 documents for antique.
First document ID: 2020338_0
First document text (snippet): A small group of politicians believed strongly that the fact that Saddam Hussien remained in power after the first Gulf War was a signal of weakness to the rest of the world, one that invited attacks ...


In [ ]:
import sys
import os

# Add the project's src directory to Python path
# تأكد من أن هذا المسار صحيح لجذر مشروعك
project_root = "C:\\Users\\FSOS\\Documents\\Projects\\ir-project"
src_path = os.path.join(project_root, 'src')

if src_path not in sys.path:
    sys.path.insert(0, src_path)


def test_suggestions(dataset_name, preprocessor):
    print(f"\n--- Testing Query Suggestions for dataset: {dataset_name} ---")

    # تهيئة خدمة اقتراح الكويري للمجموعة المختارة
    # سيقوم بتحميل الفهرس المحفوظ أو بناءه إذا لم يكن موجوداً
    suggestion_service = QuerySuggestionService(dataset_name, preprocessor)

    while True:
        user_input = input(f"Enter a query prefix for {dataset_name} (type 'exit' to quit): ").strip()
        if user_input.lower() == 'exit':
            break

        if user_input:
            suggestions_with_snippets = suggestion_service.get_suggestions(user_input, top_k=5)
            if suggestions_with_snippets:
                print("Suggestions:")
                for i, (suggestion_text, snippet_text) in enumerate(suggestions_with_snippets):
                    print(f"  {i+1}. {suggestion_text}")
                    if snippet_text:
                        print(f"     Snippet: \"{snippet_text}\"")
            else:
                print("No suggestions found.")
        else:
            print("Please enter something to get suggestions.")

# تهيئة المعالج المسبق
text_preprocessor_instance = TextPreprocessor.getInstance()

# اختبر لمجموعة البيانات "antique"
test_suggestions("antique", text_preprocessor_instance)

# اختبر لمجموعة البيانات "quora"
test_suggestions("quora", text_preprocessor_instance)

print("\nTesting complete.")

In [ ]:
import os

model_path = "data/models/all-MiniLM-L6-v2"
if os.path.exists(model_path):
    print(f"Contents of {model_path}:")
    for item in os.listdir(model_path):
        print(item)
else:
    print(f"Directory not found: {model_path}")

Contents of /content/drive/MyDrive/data/models/all-MiniLM-L6-v2:
blobs
snapshots
refs
.no_exist
